In [2]:
from scipy.io import netcdf

import iris
import iris.coord_categorisation

import os
import logging

import numpy as np

import pickle
from pathlib import Path
logger = logging.getLogger(__name__)

In [3]:
def get_scenarios():
    """
        Return a List containing SCENARIO tag names
    """
    MODELS = [ 'clm', 'jules', 'orchidee' ]
    LU_SCENARIOS = [ 'nattobiobio','nattobioaff','nattobio','nattoaff','agtonat','agtobio','agtoaff' ]
    CLIM_SCENARIOS = [ 'ssp126', 'ssp370' ]
    CLIM_MODELS = [ 'mpi-esm1-2-hr','ukesm1-0-ll','ipsl-cm6a-lr' ]
    return CLIM_SCENARIOS, CLIM_MODELS

In [26]:
def main():

    LSM = 'jules'
    var='cVeg'

    if LSM=='jules':
        SCENARIO_TITLES = ['nolucbio','nolucaff','agtobio', 'agtonat', 'agtoaff', 'nattobio', 'nattoaff']
    elif LSM=='clm' or LSM=='orchidee':
        SCENARIO_TITLES = ['noluc','agtobio', 'agtonat', 'agtoaff', 'nattobio', 'nattoaff']

    nscenarios=len(SCENARIO_TITLES)
    ssp,esm=get_scenarios()


    if LSM=='jules':
        base_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+'nolucbio.pkl')
        base_file2= Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+'nolucaff.pkl')
        with open(base_file, 'rb') as f4:
            NOLUCBIO_DICT = pickle.load(f4)
        with open(base_file2, 'rb') as f5:
            NOLUCAFF_DICT = pickle.load(f5)
    else:
        base_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+'noluc.pkl')
        with open(base_file, 'rb') as f4:
            NOLUC_DICT = pickle.load(f4)
            
    for i0 in range(nscenarios):

        print(SCENARIO_TITLES[i0])
        my_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+SCENARIO_TITLES[i0]+'.pkl')
        tropics_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+SCENARIO_TITLES[i0]+'_tropics.pkl')
        temperate_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+SCENARIO_TITLES[i0]+'_temp.pkl')

    
        if temperate_file.is_file():
            # file exists, we can skip a bunch of steps:
            with open(my_file, 'rb') as f1:
                ENSEMBLE_MEAN = pickle.load(f1)
            with open(tropics_file, 'rb') as f2:
                TROPICS_MEAN = pickle.load(f2)
            with open(temperate_file, 'rb') as f3:    
                TEMPERATE_MEAN = pickle.load(f3)

        for i1 in range(0,2):
            temparray=np.array(list(ENSEMBLE_MEAN[ssp[i1]].values()))
            if LSM=='jules':
                if SCENARIO_TITLES[i0]=='nolucbio' or SCENARIO_TITLES[i0]=='agtobio' or SCENARIO_TITLES[i0]=='nattobio':
                    nolucarray=np.array(list(NOLUCBIO_DICT[ssp[i1]].values()))
                else:
                    nolucarray=np.array(list(NOLUCAFF_DICT[ssp[i1]].values()))
            else:
                nolucarray=np.array(list(NOLUC_DICT[ssp[i1]].values()))
            
            delta_gb=(temparray[:,0,-1]-temparray[:,0,0]) - (nolucarray[:,0,-1]-nolucarray[:,0,0]) 
           
        
            ssp_mean = np.mean(delta_gb)
            ssp_min = np.min(delta_gb)
            ssp_max = np.max(delta_gb)
        
            print('Global average,min,max change for:',ssp[i1],ssp_mean,ssp_min,ssp_max)
        print(' ')
        

In [27]:
if __name__ == "__main__":
    main()

nolucbio
bioscenario
Global average,min,max change for: ssp126 0.0 0.0 0.0
bioscenario
Global average,min,max change for: ssp370 0.0 0.0 0.0
 
nolucaff
Global average,min,max change for: ssp126 0.0 0.0 0.0
Global average,min,max change for: ssp370 0.0 0.0 0.0
 
agtobio
bioscenario
Global average,min,max change for: ssp126 -0.5339245128253651 -0.6346323536297405 -0.4523922732238361
bioscenario
Global average,min,max change for: ssp370 -0.9086521440408433 -1.0195911427881583 -0.791503099031047
 
agtonat
Global average,min,max change for: ssp126 1.540742486102393 1.4095020051728397 1.6454201160318007
Global average,min,max change for: ssp370 1.7224580191257246 1.5629069912433806 1.8575763848743918
 
agtoaff
Global average,min,max change for: ssp126 1.644286927316254 1.4886966586103654 1.76436559538638
Global average,min,max change for: ssp370 1.8285251274464827 1.6383099838428734 1.9931263458422563
 
nattobio
bioscenario
Global average,min,max change for: ssp126 -44.98730625651327 -45.194